In [3]:
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

GIGACHAT_API_KEY = "NzM3NmE5NGQtNzBlNi00NGI2LWI3NjQtMTBlYzc3NmMwZjU1OmMwOGM4YTRmLTA5ZjgtNGFhNS05YjlkLTZjYTdlMmIwNWUyNA=="

class Agent:
    def __init__(self, role_prompt: str):
        
        self.payload = Chat(
            messages=[
                Messages(
                    role=MessagesRole.SYSTEM,
                    content=role_prompt,
                )
            ],
            temperature=0.3,
        )
        
    def query(self, prompt: str):
        with GigaChat(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False) as giga:
            self.payload.messages.append(
                Messages(
                    role=MessagesRole.USER,
                    content=prompt,
                )
            )
            
            response = giga.chat(self.payload)
            self.payload.messages.append(response.choices[0].message)
            return response.choices[0].message.content
        
        

class Environment:
    def __init__(self, 
                 primary_role: str,
                 secondary_role_1: str,
                 secondary_role_2: str
                ):
        
        self.primary_agent     = Agent(primary_role)
        self.secondary_agent_1 = Agent(secondary_role_1)
        self.secondary_agent_2 = Agent(secondary_role_2)
        
    def dialog_episode(self, secondary_prompt_1: str, secondary_prompt_2: str, test=False):
        response_agent_1 = self.secondary_agent_1.query(secondary_prompt_1)
        response_agent_2 = self.secondary_agent_2.query(secondary_prompt_2)
        
        response_primary_raw   = self.primary_agent.query(f'прокурор: "{response_agent_1}"')
        response_primary_final = self.primary_agent.query(f'адвокат: "{response_agent_2}"')
        
        if test:
            print(f" прокурор: {response_agent_1}\n судья: {response_primary_raw}\n адвокат: {response_agent_2}\n судья: {response_primary_final}")
        
        return response_primary_raw, response_primary_final, response_agent_1, response_agent_2
    
    def get_order(self):
        response_primary_final = self.primary_agent.query("Комада: вынести решение")
        
        return response_primary_final
        

In [2]:
primary_role = 'Ты судья гражданского суда в России и ведешь судебный процесс. Ты по очереди выслушиваешь стороны обвинителя (прокурора) и защиты (адвоката). Например: прокурор: "Уважаемый суд! Сегодня я, как представитель прокуратуры, выступаю в защиту законных интересов гражданина РФ Ивана Иванова". После того, как обе стороны высказались ты при необходимости можешь задать им вопросы. Тебе необходимо вынести решение суда после кодовой фразы "Комада: вынести решение"'
secondary_role_1 = "Ты прокурор гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"
secondary_role_2 = "Ты адвокат гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"

court = Environment(primary_role, secondary_role_1, secondary_role_2)

In [3]:
secondary_prompt_1 = "Твоя задача взыскать средства объемом в 300 тысяч рублей с гражданина Иванан Иванова в пользу гражданина Петра Петровича в связи с тем, что Иван Иванов нарушил права Петра Петровича. А именно Иван Иванов испортил имующество пренадлежащее Петру Петровичу."
secondary_prompt_2 = "Твоя задача защитить интересы гражданина Иванан Иванова, которого гражданин Петр Петрович ложно обвинил в порче его имущества на сумму в 300 тысяч рублей."

a,b = court.dialog_episode(secondary_prompt_1, secondary_prompt_2)

In [4]:
print(a)

Хорошо, господин прокурор, вы можете начать ваше выступление.


In [5]:
print(b)

Спасибо, защита. Теперь давайте послушаем сторону обвинения. Прошу вас, господин прокурор.


In [6]:
secondary_prompt_1 = "настаивай на том, что у Ивана Иванова нет доказательтв невиновности"
secondary_prompt_2 = "судья спрашивает у тебя: какие доказательства вы можете представить суду в подтверждение невиновности Иванан Иванова?"
a, b, c, d = court.dialog_episode(secondary_prompt_1, secondary_prompt_2)

In [7]:
print(a)

Понятно. Господин прокурор, вы можете представить суду доказательства вашей позиции?


In [8]:
print(b)

Благодарю за информацию. Я учту все представленные вами данные при вынесении решения. Есть ли еще какие-либо сведения или аргументы, которые вы хотели бы представить суду?


In [9]:
print(court.get_order())

Исходя из представленных доказательств и аргументов сторон, я выношу следующее решение: признать гражданина Иванова невиновным в причинении ущерба имуществу гражданина Ретровича.


In [20]:
import tkinter as tk
from tkinter import ttk, scrolledtext

class ChatInterface:
    def __init__(self, master):
        self.master = master
        master.title("Диалоговый интерфейс")
        
        # Хранилища для промптов
        self.secondary_prompt_1 = ""  # Для прокурора
        self.secondary_prompt_2 = ""  # Для адвоката
        
        # Основной текстовый виджет
        self.history_area = scrolledtext.ScrolledText(master, wrap=tk.WORD, state='disabled')
        self.history_area.pack(padx=10, pady=7, fill=tk.BOTH, expand=True)
        
        # Панель управления
        control_frame = ttk.Frame(master)
        control_frame.pack(pady=5, fill=tk.X)
        
        # Кнопки действий
        self.court_btn = ttk.Button(control_frame, text="Решение Суда", command=self.show_court_order)
        self.court_btn.pack(side=tk.LEFT, padx=2)
        
        self.get_btn = ttk.Button(control_frame, text="Получить", command=self.get_responses)
        self.get_btn.pack(side=tk.LEFT, padx=2)
        
        self.send_btn = ttk.Button(control_frame, text="Отправить", command=self.send_message)
        self.send_btn.pack(side=tk.LEFT, padx=2)
        
        # Переключатель режима
        self.mode_var = tk.StringVar(value="prosecutor")
        mode_switch = ttk.Frame(master)
        mode_switch.pack(pady=5)
        
        ttk.Radiobutton(mode_switch, text="Прокурор", variable=self.mode_var, 
                      value="prosecutor", command=self.update_prompts).pack(side=tk.LEFT, padx=5)
        ttk.Radiobutton(mode_switch, text="Адвокат", variable=self.mode_var, 
                      value="lawyer", command=self.update_prompts).pack(side=tk.LEFT, padx=5)
        
        # Поле ввода
        self.input_field = ttk.Entry(master)
        self.input_field.pack(padx=20, pady=10, fill=tk.X)
        self.input_field.bind("<Return>", lambda e: self.send_message())
        
        # Инициализируем начальное значение поля ввода
        self.update_prompts()

    def update_prompts(self):
        """Обновление поля ввода в соответствии с выбранным режимом"""
        current_mode = self.mode_var.get()
        if current_mode == "prosecutor":
            self.input_field.delete(0, tk.END)
            self.input_field.insert(0, self.secondary_prompt_1)
        else:
            self.input_field.delete(0, tk.END)
            self.input_field.insert(0, self.secondary_prompt_2)

    def update_history(self, text):
        """Обновление истории"""
        self.history_area.configure(state='normal')
        self.history_area.insert(tk.END, text + "\n\n")
        self.history_area.configure(state='disabled')
        self.history_area.see(tk.END)

    def send_message(self):
        """Отправка сообщения"""
        message = self.input_field.get()
        if message:
            # Сохраняем в соответствующий промпт
            if self.mode_var.get() == "prosecutor":
                self.secondary_prompt_1 = message
            else:
                self.secondary_prompt_2 = message
            
            self.update_history(f"Пользователь ({self.mode_var.get()}): {message}")
            self.input_field.delete(0, tk.END)

    def get_responses(self):
        """Получение ответов системы"""
        # Сохраняем текущее сообщение перед получением ответа
        current_message = self.input_field.get()
        if current_message:
            self.send_message()  # Сохраняем сообщение в соответствующий промпт

        # Вызываем диалог с сохраненными промптами
        r_raw, r_final, r_agent1, r_agent2 = court.dialog_episode(
            secondary_prompt_1=self.secondary_prompt_1,
            secondary_prompt_2=self.secondary_prompt_2
        )
        
        response = (f"Прокурор: {r_agent1}\nАдвокат: {r_agent2}\n"
                    f"Система:\nRAW: {r_raw}\nFinal: {r_final}\n"                
                   )
        
        self.update_history(response)

    def show_court_order(self):
        """Показать решение суда"""
        order = court.get_order()
        self.update_history(f"Суд принял решение: {order}")



primary_role = 'Ты судья гражданского суда в России и ведешь судебный процесс. Ты по очереди выслушиваешь стороны обвинителя (прокурора) и защиты (адвоката). Например: прокурор: "Уважаемый суд! Сегодня я, как представитель прокуратуры, выступаю в защиту законных интересов гражданина РФ Ивана Иванова". После того, как обе стороны высказались ты при необходимости можешь задать им вопросы. Тебе необходимо вынести решение суда после кодовой фразы "Комада: вынести решение"'
secondary_role_1 = "Ты прокурор гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"
secondary_role_2 = "Ты адвокат гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"

court = Environment(primary_role, secondary_role_1, secondary_role_2)

if __name__ == "__main__":
    root = tk.Tk()
    app = ChatInterface(root)
    root.geometry("1000x600")
    root.mainloop()

In [6]:
import tkinter as tk
from tkinter import ttk, scrolledtext

class ChatInterface:
    def __init__(self, master):
        self.master = master
        master.title("Диалоговый интерфейс")
        self.secondary_prompt_2
        self.secondary_prompt_1
        # Основной текстовый виджет с прокруткой
        self.history_area = scrolledtext.ScrolledText(master, wrap=tk.WORD, state='disabled')
        self.history_area.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Панель управления
        control_frame = ttk.Frame(master)
        control_frame.pack(pady=5, fill=tk.X)
        
        # Кнопки действий
        self.court_btn = ttk.Button(control_frame, text="Решение Суда", command=self.show_court_order)
        self.court_btn.pack(side=tk.LEFT, padx=2)
        
        self.get_btn = ttk.Button(control_frame, text="Получить", command=self.get_responses)
        self.get_btn.pack(side=tk.LEFT, padx=2)
        
        self.send_btn = ttk.Button(control_frame, text="Отправить", command=self.send_message)
        self.send_btn.pack(side=tk.LEFT, padx=2)
        
        # Переключатель режима
        self.mode_var = tk.StringVar(value="prosecutor")
        mode_switch = ttk.Frame(master)
        mode_switch.pack(pady=5)
        
        ttk.Radiobutton(mode_switch, text="Прокурор", variable=self.mode_var, 
                      value="prosecutor").pack(side=tk.LEFT, padx=5)
        ttk.Radiobutton(mode_switch, text="Адвокат", variable=self.mode_var, 
                      value="lawyer").pack(side=tk.LEFT, padx=5)
        
        # Поле ввода сообщения
        self.input_field = ttk.Entry(master)
        self.input_field.pack(padx=10, pady=10, fill=tk.X)
        self.input_field.bind("<Return>", lambda e: self.send_message())
    
    def update_history(self, text):
        """Обновление истории диалога"""
        self.history_area.configure(state='normal')
        self.history_area.insert(tk.END, text + "\n\n")
        self.history_area.configure(state='disabled')
        self.history_area.see(tk.END)
    
    def send_message(self):
        """Отправка сообщения из поля ввода"""
        message = self.input_field.get()
        if 
        if message:
            self.update_history(f"Пользователь: {message}")
            self.input_field.delete(0, tk.END)
    
    def get_responses(self):
        """Получение ответов от системы"""
        message = self.input_field.get()
        if message:
            # Вызов функции диалога
            r_raw, r_final, r_agent1, r_agent2 = dialog_episode(
                secondary_prompt_1=message,
                secondary_prompt_2=self.mode_var.get()
            )
            
            # Формирование ответа
            response = (f"Система:\nRAW: {r_raw}\nFinal: {r_final}\n"
                       f"Agent 1: {r_agent1}\nAgent 2: {r_agent2}")
            
            self.update_history(response)
            self.input_field.delete(0, tk.END)
    
    def show_court_order(self):
        """Показать решение суда"""
        order = get_order()
        self.update_history(f"Суд принял решение: {order}")

primary_role = 'Ты судья гражданского суда в России и ведешь судебный процесс. Ты по очереди выслушиваешь стороны обвинителя (прокурора) и защиты (адвоката). Например: прокурор: "Уважаемый суд! Сегодня я, как представитель прокуратуры, выступаю в защиту законных интересов гражданина РФ Ивана Иванова". После того, как обе стороны высказались ты при необходимости можешь задать им вопросы. Тебе необходимо вынести решение суда после кодовой фразы "Комада: вынести решение"'
secondary_role_1 = "Ты прокурор гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"
secondary_role_2 = "Ты адвокат гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"

court = Environment(primary_role, secondary_role_1, secondary_role_2)

if __name__ == "__main__":
    root = tk.Tk()
    app = ChatInterface(root)
    root.geometry("800x600")
    root.mainloop()

In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext

def get_decision():
    prosecutor_text = prosecutor_entry.get("1.0", tk.END).strip()
    lawyer_text = lawyer_entry.get("1.0", tk.END).strip()
    
    result1 = "----- Решение -----\n"
    result1 += court.get_order()
    
    output_text.config(state=tk.NORMAL)
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.INSERT, result1)
    output_text.config(state=tk.DISABLED)




def say_arg():
    secondary_prompt_1 = prosecutor_entry.get("1.0", tk.END).strip()
    secondary_prompt_2 = lawyer_entry.get("1.0", tk.END).strip()
    
    result = "Ваши Доводы учтены\n"
    
    response_primary_raw, response_primary_final, response_agent_1, response_agent_2 = court.dialog_episode(secondary_prompt_1, secondary_prompt_2)
    

    result = result + response_agent_1 + "\n" + response_agent_2 + "\n"
    
    output_text.config(state=tk.NORMAL)
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.INSERT, result)
    output_text.config(state=tk.DISABLED)

    


# Создание главного окна
root = tk.Tk()
root.title("Судебное решение")
root.geometry("1000x680")



primary_role = 'Ты судья гражданского суда в России и ведешь судебный процесс. Ты по очереди выслушиваешь стороны обвинителя (прокурора) и защиты (адвоката). Например: прокурор: "Уважаемый суд! Сегодня я, как представитель прокуратуры, выступаю в защиту законных интересов гражданина РФ Ивана Иванова". После того, как обе стороны высказались ты при необходимости можешь задать им вопросы. Тебе необходимо вынести решение суда после кодовой фразы "Комада: вынести решение"'
secondary_role_1 = "Ты прокурор гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"
secondary_role_2 = "Ты адвокат гражданского суда в России. Ты выслушиваешь описание материала дела и выступаешь по нему перед судом, отвечая на вопросы судьи если тебе их зададут"

court = Environment(primary_role, secondary_role_1, secondary_role_2)



# Стиль для меток
style = ttk.Style()
style.configure("TLabel", font=('Arial', 10, 'bold'), padding=5)

# Текстовые поля с прокруткой
def create_scrolled_text(frame, height=10):
    text_area = scrolledtext.ScrolledText(frame, wrap=tk.WORD, height=height, font=('Arial', 10))
    return text_area

# Фрейм для прокурора
prosecutor_frame = ttk.Frame(root)
prosecutor_frame.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)
ttk.Label(prosecutor_frame, text="Прокурор:").pack(anchor=tk.W)
prosecutor_entry = create_scrolled_text(prosecutor_frame)
prosecutor_entry.pack(fill=tk.BOTH, expand=True)

# Фрейм для адвоката
lawyer_frame = ttk.Frame(root)
lawyer_frame.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)
ttk.Label(lawyer_frame, text="Адвокат:").pack(anchor=tk.W)
lawyer_entry = create_scrolled_text(lawyer_frame)
lawyer_entry.pack(fill=tk.BOTH, expand=True)

# Кнопка
button_frame = ttk.Frame(root)
button_frame.pack(pady=10)
decision_btn = ttk.Button(button_frame, text="Сказать Судъе", command=say_arg) ## вот тут писать чо-то
decision_btn.pack()

# button 2
button_frame = ttk.Frame(root)
button_frame.pack(pady=10)
decision_btn = ttk.Button(button_frame, text="Получить решение Суда", command=get_decision) ## вот тут писать чо-то
decision_btn.pack()

# Поле вывода
output_frame = ttk.Frame(root)
output_frame.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)
ttk.Label(output_frame, text="Уважаемый Суд:").pack(anchor=tk.W)
output_text = scrolledtext.ScrolledText(output_frame, wrap=tk.WORD, height=10, font=('Arial', 10))
output_text.pack(fill=tk.BOTH, expand=True)
output_text.config(state=tk.DISABLED)

root.mainloop()